In [2]:
from pathlib import Path

In [7]:
import json

emails_raw = Path("~/Downloads/emails.json").expanduser().read_text()
emails = json.loads(emails_raw)


In [12]:
emails[0]

def format_email(email):
    return f"From: {email['from']}\n" \
        f"To: {email['to']}\n" \
        f"Subject: {email['subject']}\n" \
        f"Body (snippet): {email['snippet']}"

emails_formatted = [format_email(email) for email in emails]

In [22]:
import openai

api_key = Path("~/.openai").expanduser().read_text()

client = openai.OpenAI(api_key=api_key)

embeddings = client.embeddings.create(
    model="text-embedding-3-large",
    input=emails_formatted,
)

In [ ]:
import numpy as np

embeddings_np = np.array([embedding.embedding for embedding in embeddings.data])

embeddings_np.shape


In [ ]:
# Apply PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
embeddings_pca = pca.fit_transform(embeddings_np)

embeddings_pca.shape


In [ ]:
# Run k-means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(embeddings_pca)

kmeans.labels_


In [ ]:
# Sample first 3 from each cluster
emails_formatted_np = np.array(emails_formatted)
for cluster_id in range(3):
    print(f"Cluster {cluster_id}:")
    for i in range(3):
        print(emails_formatted_np[kmeans.labels_ == cluster_id][i])
